In [ ]:
# |default_exp download

# Constantes

In [ ]:
# |export

AUDIO_PATH = "audios"

# collection

In [ ]:
from connection import get_collection

collection = get_collection()

# get_audio_path

In [ ]:
# |export

import os
from git import Repo


def get_audio_path(audio_path=AUDIO_PATH):
    """
    return full audio path and create dir if it doesn t exist
    """

    def get_git_root(path):
        git_repo = Repo(path, search_parent_directories=True)
        return git_repo.git.rev_parse("--show-toplevel")

    project_root = get_git_root(os.getcwd())
    full_audio_path = os.path.join(project_root, audio_path)

    # create dir if it doesn t exist
    if not os.path.exists(full_audio_path):
        os.makedirs(full_audio_path)

    return full_audio_path

In [ ]:
get_audio_path()

'/home/guillaume/git/lmelp/audios'

In [ ]:
get_audio_path("audio_temp")

'/home/guillaume/git/lmelp/audio_temp'

# download mp3 livres

In [ ]:
# |export

import requests
from bson import ObjectId


def download_mp3_livres(collection, oid, audio_path=AUDIO_PATH, verbose=False):
    """
    on telecharge le fichier mp3 du document avec l oid specifie
    si :
    - le champ type vaut livres
    - le champ url existe
    - le champ duree existe et est superieur a 15 minutes (ajouter un parametrage apres)
    - le mp3 n existe pas deja dans audio_path

    si on veut forcer le telechargement, il faut manuellement supprimer le fichier mp3 dans audio_path

    collection: pymongo collection
    oid: str, oid du document
    audio_path: str, path to save audio files, default is 'AUDIO_PATH', abreviated, and is created if it doesn t exist
    """

    full_audio_path = get_audio_path(audio_path)
    if verbose:
        print(f"audio path: {full_audio_path}")

    # Récupération du document
    document_entry = collection.find_one({"_id": ObjectId(oid)})

    if document_entry is None:
        if verbose:
            print(f"Document avec l'oid {oid} non trouvé")
        return

    # Vérification si le type est bien "livres"
    if document_entry.get("type") != "livres":
        if verbose:
            print(f"Document avec l'oid {oid} n'est pas de type livres")
        return

    # verfiication si la duree est > 15 minutes
    duree = document_entry.get("duree")
    if duree is None or duree < 15:
        if verbose:
            print(f"Document avec l'oid {oid} n'est fait moins de 15 minutes")
        return

    url = document_entry.get("url")
    if url is None:
        if verbose:
            print(f"Document avec l'oid {oid} n'a pas d'url")
        return
    if verbose:
        print(f'{document_entry["type"]} - {document_entry.get("description")} - {url}')

    filename = os.path.join(full_audio_path, os.path.basename(url))

    # Vérification si le fichier existe déjà
    if not os.path.exists(filename):
        if verbose:
            print(f"Téléchargement de {url} vers {filename}")
        response = requests.get(url)
        with open(filename, "wb") as file:
            file.write(response.content)
    else:
        if verbose:
            print(f"Le fichier {filename} existe déjà. Ignoré.")

In [ ]:
download_mp3_livres(collection, "6773e32258fc5717f3516ba0", verbose=True)

audio path: /home/guillaume/git/lmelp/audios
Document avec l'oid 6773e32258fc5717f3516ba0 n'est pas de type livres


# download all mp3 livres

In [ ]:
# |export

import requests


def download_all_mp3_livres(collection, audio_path=AUDIO_PATH, verbose=False):
    """
    on boucle dans tous les documents de la collection et telecharge les fichiers mp3
    si :
    - le champ type vaut livres
    - le champ url existe
    - le champ duree existe et est superieur a 15 minutes (ajouter un parametrage apres)
    - le mp3 n existe pas deja dans audio_path

    si on veut forcer le telechargement, il faut manuellement supprimer le fichier mp3 dans audio_path

    collection: pymongo collection
    audio_path: str, path to save audio files, default is 'AUDIO_PATH', abreviated, and is created if it doesn t exist
    """

    full_audio_path = get_audio_path(audio_path)
    if verbose:
        print(f"audio path: {full_audio_path}")
    # Récupération de toutes les entrées dans la collection
    entries = collection.find()

    for entry in entries:

        # get oid
        oid = entry["_id"]
        if verbose:
            print(oid)

        download_mp3_livres(collection, oid, audio_path, verbose)

In [ ]:
download_all_mp3_livres(collection, verbose=True)

audio path: /home/guillaume/git/lmelp/audios
6773e32258fc5717f3516b98
audio path: /home/guillaume/git/lmelp/audios
Document avec l'oid 6773e32258fc5717f3516b98 n'est pas de type livres
6773e32258fc5717f3516b99
audio path: /home/guillaume/git/lmelp/audios
livres - durée : 00:47:52 - Le Masque et la Plume - par : Rebecca Manzoni - Il est encore temps d'ajouter quelques livres sous le sapin - invités : Raphaelle Leyris, Jean-Marc Proust, Patricia Martin, Laurent CHALUMEAU - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Jean-Marc Proust : Auteur et critique (Slate), Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier - réalisé par : Guillaume Girault - https://rf.proxycast.org/7e653bf4-87a5-42f4-864b-9208e206a295/14007-22.12.2024-ITEMA_23973143-2024F4007S0357-22.mp3
Le fichier /home/guillaume/git/lmelp/audios/14007-22.12.2024-ITEMA_23973143-2024F4007S0357-22.mp3 existe 

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("08 download mp3 from db info.ipynb", ".")